In [1]:
from math import log, sqrt, exp
import time
import datetime as dt                   # date objects
import numpy as np                      # array manipulation
import matplotlib.pyplot as plot        # plotting
import pandas as pd                     # data analysis
import pandas_datareader as pdr
from scipy.stats import norm  # normal cdf

from ib_insync import *
util.startLoop()

#### Trading Assignment 1

Go to https://finance.yahoo.com and download adjusted close daily prices (end of trading day prices adjusted for dividends and splits) for the period 03/15/2022 to 09/15/2022 for SPY ETF.

In [2]:
# pull data from 
source = 'yahoo'
indicators = ['VOO']
end_date = dt.date.today()
start_date = end_date - dt.timedelta(days=365)
raw_data = pdr.DataReader(indicators, source, start_date, end_date)
data = raw_data['Adj Close']

1. Calculate the annual volatility for the S&P500 index. Use 252 trading days.

In [3]:
std_SPY = data.tail(252).VOO.std()
mean_SPY = data.tail(252).VOO.mean()
print(f"Annual Volatility SPY: {std_SPY}")

Annual Volatility SPY: 25.082680765672922


2. Go to https://www.cboe.com/delayed_quotes/spy/quote_table and select five option contracts that are nearest to ATM with expiration as close as possible to 30 days and have open interest and volume of at least 1,000 contracts. Alternatively, connect to IB and download the option chain. (this part can be done with a bot)
Using the volatility, you calculated in (1) and the Black-Scholes model, compute the “theoretical” prices of these call options.

In [4]:
try:
    ib.disconnect()
    time.sleep(5)
except:
    print("no ib connection to disconnect")

# initialize connection to IBKR
ib = IB()
ib.connect('127.0.0.1', 7497, clientId=1)  # IB Trader Workstation
#ib.connect('127.0.0.1', 4002, clientId=1)    # IB Gateway

no ib connection to disconnect


<IB connected to 127.0.0.1:7497 clientId=1>

In [5]:
# create a contract for the underlying s&p500 index
spx = Stock('VOO', 'SMART', 'USD')
ib.qualifyContracts(spx)
ib.reqMarketDataType(4)

# grab the ticker
[ticker] = ib.reqTickers(spx)
spx_price = ticker.marketPrice()

In [6]:
# generate the option chain
opt_chain = ib.reqSecDefOptParams(spx.symbol, '', spx.secType, spx.conId)
chain = next(c for c in opt_chain if c.tradingClass == 'VOO' and c.exchange == 'SMART')

# filter strikes for those close to in the money
strikes = [s for s in chain.strikes
           if spx_price - 20 < s < spx_price + 20]

expirations = sorted(exp for exp in chain.expirations)[:2]

rights = ['C']

contracts = [Option('VOO', expiration, strike, right, 'SMART', tradingClass='VOO')
        for right in rights
        for expiration in expirations
        for strike in strikes]
contracts = ib.qualifyContracts(*contracts)
#ib.reqMarketDataType(4)

In [7]:
contracts[:10]

[Option(conId=573941383, symbol='VOO', lastTradeDateOrContractMonth='20220916', strike=345.0, right='C', multiplier='100', exchange='SMART', currency='USD', localSymbol='VOO   220916C00345000', tradingClass='VOO'),
 Option(conId=573941418, symbol='VOO', lastTradeDateOrContractMonth='20220916', strike=350.0, right='C', multiplier='100', exchange='SMART', currency='USD', localSymbol='VOO   220916C00350000', tradingClass='VOO'),
 Option(conId=573941458, symbol='VOO', lastTradeDateOrContractMonth='20220916', strike=355.0, right='C', multiplier='100', exchange='SMART', currency='USD', localSymbol='VOO   220916C00355000', tradingClass='VOO'),
 Option(conId=573941493, symbol='VOO', lastTradeDateOrContractMonth='20220916', strike=360.0, right='C', multiplier='100', exchange='SMART', currency='USD', localSymbol='VOO   220916C00360000', tradingClass='VOO'),
 Option(conId=573941535, symbol='VOO', lastTradeDateOrContractMonth='20220916', strike=365.0, right='C', multiplier='100', exchange='SMART',

In [8]:
# grab an example contract
call = contracts[8]
[ticker] = ib.reqTickers(call)
ticker

Ticker(contract=Option(conId=545580776, symbol='VOO', lastTradeDateOrContractMonth='20221021', strike=345.0, right='C', multiplier='100', exchange='SMART', currency='USD', localSymbol='VOO   221021C00345000', tradingClass='VOO'), time=datetime.datetime(2022, 9, 14, 17, 26, 19, 991759, tzinfo=datetime.timezone.utc), marketDataType=3, bid=22.5, bidSize=3357.0, ask=23.1, askSize=1252.0, last=0.0, lastSize=0.0, volume=0.0, close=21.75, bidGreeks=OptionComputation(tickAttrib=0, impliedVol=0.25923765122873504, delta=0.7470886959288471, optPrice=22.5, pvDividend=1.5961177143034238, gamma=0.011484208404046686, vega=0.38133644286133617, theta=-0.15886094266356568, undPrice=362.9100036621094), askGreeks=OptionComputation(tickAttrib=0, impliedVol=0.27749564667282656, delta=0.7334455492548493, optPrice=23.100000381469727, pvDividend=1.5961177143034238, gamma=0.010893225376620458, vega=0.3823187223720552, theta=-0.17037651665547965, undPrice=362.9100036621094), lastGreeks=OptionComputation(tickAttr

In [9]:
# now calculate the theoretical prices of these options using BS

def bs_price(S, K, T, r, sigma, opt="c"):
    """Price an option with the black-scholes model

    Args:
        S (float): Price of the underlying security
        K (float): Strike price
        T (float): time to maturity
        r (float): risk free interest rate
        sigma (float): volatility of the underlying security
        opt (str, optional): option type. Defaults to "C" (call)

    Returns:
        float: the black-scholes price of the option
    """
    d1 = (np.log(S / K) + (r + sigma**2 / 2.0) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)

    if opt == "c":
        bs_price = S * norm.cdf(d1) - K * np.exp(-r * T) * norm.cdf(d2)
    elif opt == "p":
        bs_price = K * np.exp(-r * T) * norm.cdf(-d2) - S * norm.cdf(-d1)
    else:
        bs_price = None
    return bs_price


Taking an example contract, we can compare the price with the Black-Scholes price. Do this for each contract in your list and find the one with the biggest discrepancy (hint: list comprehension is useful here!). For our purposes we'll just grab the first one.

In [10]:
price = spx_price
strike = ticker.contract.strike
maturity = 37/365
rfr = 0.03
vol = std_SPY / mean_SPY

opt_price = bs_price(price, strike, maturity, rfr, vol)

print(f"The close price of the option is: {ticker.close}")
print(f"The black-scholes price of the option is: {opt_price}")

The close price of the option is: 21.75
The black-scholes price of the option is: 19.006499729112534


**Note:** We'll continue parts 3-4 by wrapping what we have done so far into a class that we can periodically instantiate and run it from [ta1.py](./ta1.py) 

3. At the beginning of the first trading day, sell 10 contracts of the call option with the largest mispricing. What would the proceeds from your trade be if the market price equals the BS option value? What are the actual proceeds from your trade?

In [11]:
# collect our option contract and make our market order
contr = call
order = MarketOrder('SELL', 10)

trade = ib.placeOrder(contr, order)

# some assertions to ensure our trade has been placed
assert order in ib.orders()
assert trade in ib.trades()

while not trade.isDone():
    ib.waitOnUpdate()
    
ib.positions()

[Position(account='DU6059326', contract=Option(conId=545580776, symbol='VOO', lastTradeDateOrContractMonth='20221021', strike=345.0, right='C', multiplier='100', currency='USD', localSymbol='VOO   221021C00345000', tradingClass='VOO'), position=-10.0, avgCost=2249.8897145)]

4. As soon as possible after executing the trade, you initiate a delta hedge for your options. The hedge will be rebalanced (adjusted) during the next four weeks, i.e., until the expiation date or until the hedged position is closed. \n At initiation of your hedge, what is the delta for the call option? How many shares do you have to trade? Considering the proceeds from the options sale, how much would you have to you have to borrow (in real life) to finance a delta-neutral position. What is the interest cost incurred at the end of the first trading day?

Alright, let's delta hedge! The option delta is simply the sensitivity of the option to a change in price in the underlying security. Mathematically that is represented as...

$$
delta = \frac{\partial V}{\partial S}
$$

So we can simply take the derivative of the black scholes function we have above...

In [12]:
def bs_delta(S, K, T, r, sigma, opt="C"):
    """Calculate the black-scholes delta of the option

    Args:
        S (float): Price of the underlying security
        K (float): Strike price
        T (float): time to maturity
        r (float): risk free interest rate
        sigma (float): volatility of the underlying security
        opt (str, optional): option type. Defaults to "C" (call)

    Returns:
        float: the black-scholes delta of the option
    """
    d1 = (np.log(S / K) + (r + sigma**2 / 2.0) * T) / (sigma * np.sqrt(T))

    if opt == "C":
        bs_delta = norm.cdf(d1)
    elif opt == "P":
        bs_delta = -norm.cdf(-d1)
    else:
        bs_delta = None
    return bs_delta


In [13]:
# let's compare this to the delta provided by the IB modelGreeks functionality
opt_delta = bs_delta(price, strike, maturity, rfr, vol)

print(f"The delta of the option is: {ticker.modelGreeks.delta}")
print(f"The black-scholes delta of the option is: {opt_delta}")

The delta of the option is: 0.7434154199183308
The black-scholes delta of the option is: 0.996103025578386


In [14]:
# get current positions
positions = [p for p in ib.positions() if p.contract.symbol == "VOO"]

# ensure that you have an open option position
opts = [p for p in positions if p.contract.secType == "OPT"]
if not opts:
    print("No option position to hedge")
option = opts[0]

delta = ticker.modelGreeks.delta
delta_neutral_pos = round(-option.position * delta * 100)

# calculate the difference between the hedge requirement and the current position
stks = [p for p in positions if p.contract.secType == "STK"]
if stks:
    hedge = delta_neutral_pos - stks[0].position
else:
    hedge = delta_neutral_pos


In [15]:
# TODO: only execute the trade if the hedge outweighs the risk
if hedge > 0:
    # return contract and order required to remain delta neutral
    contract = Stock("VOO", "SMART", "USD")
    order = MarketOrder("BUY", hedge)

    # place trade
    trade = ib.placeOrder(contract, order)
    assert order in ib.orders()
    assert trade in ib.trades()
    while not trade.isDone():
        ib.waitOnUpdate()
else:
    print("Already delta-neutral, no trade required.")

In [16]:
ib.positions()

[Position(account='DU6059326', contract=Option(conId=545580776, symbol='VOO', lastTradeDateOrContractMonth='20221021', strike=345.0, right='C', multiplier='100', currency='USD', localSymbol='VOO   221021C00345000', tradingClass='VOO'), position=-10.0, avgCost=2249.2471450000003),
 Position(account='DU6059326', contract=Stock(conId=136155102, symbol='VOO', exchange='ARCA', currency='USD', localSymbol='VOO', tradingClass='VOO'), position=743.0, avgCost=362.9373082099596)]

5. At least twice during each trading day for the next four weeks, rebalance your portfolio so that your risk exposure remains as close to delta neutral as possible. Remember transaction costs! Do not trade unless the stock has moved enough, i.e. more than a round-trip transaction costs!  (this part can be done with an algo)

6. In four weeks, the options are either exercised or expire with 0 value or the hedged position is closed. What is the value of your delta neutral position? Did you make or lost money? Explain why?

In [17]:
ib.disconnect()